In [2]:
import pandas as pd
import re
log = pd.read_csv('visit_log.csv', sep=';')
url = pd.read_csv('URLs.txt')
rating = pd.read_csv('ml-latest-small/ratings.csv')

In [3]:
log.loc[(log.traffic_source == 'google') | (log.traffic_source == 'yandex'), 'source_type'] = 'organic'
log.loc[(log.traffic_source == 'paid') & (log.region == 'Russia'), 'source_type'] = 'ad'
log.loc[(log.traffic_source == 'paid') & (log.region != 'Russia'), 'source_type'] = 'other'
log.loc[(log.traffic_source == 'email'), 'source_type'] = 'email'
log.loc[(log.traffic_source == 'direct'), 'source_type'] = 'direct'
log.head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,ad
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,direct
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,direct
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,email


In [4]:
url[url.url.str.contains(''r'/\d{8}')].head(10)

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
10,/middleeast/36131117-divizion-s-400-ne-zametil...
11,/economics/36065674-rossiyane-vozmutilis-minim...
12,/politics/36118047-otets-24-detey-potreboval-i...
13,/travel/36194479-v-sheremetevo-passazhiry-ustr...
14,/politics/35638742-nesolidno-mariya-zaharova-s...


In [104]:
rating_sort = (
    rating
    .groupby('userId') #Группировка по ID
    .agg(
        {'timestamp': ['min', 'max'], #Столбцы  min, max и count
         'rating': ['count']})
    [rating_sort['rating']['count'] >= 100]) #количество оценок больше 100
rating_sort

timestamp             rating
               min         max  count
userId                               
4        949778714   949982274    204
5       1163373044  1163375145    100
8       1154389340  1154474527    116
15       997937239  1469330735   1700
17      1127468587  1127476640    363
...            ...         ...    ...
656      986240991   986244044    128
659      834598040   866207451    142
664     1343731283  1441911722    519
665      992836298  1046967769    434
671     1063500751  1074784735    115

[263 rows x 3 columns]

In [101]:
lt = rating_sort['timestamp']['max'] - rating_sort['timestamp']['min']
lt.mean()

39331748.56653992

In [105]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [108]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [109]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [110]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [119]:
(client_base
    .merge(rzd, on = 'client_id', how = 'outer')
    .merge(auto, on = 'client_id', how = 'outer')
    .merge(air, on = 'client_id', how = 'outer')
)

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


In [122]:
(rzd
    .merge(auto, on = 'client_id', how = 'outer')
    .merge(air, on = 'client_id', how = 'outer')
)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0
